# ORPO微調整の例

このノートブックでは、`trl`ライブラリを使用してORPO（選好確率比最適化）を実行する方法を示します。ORPOは、モデルの出力を人間の選好に合わせるためのシンプルで効果的な方法です。

## 1. 環境の設定

まず、必要なライブラリをインストールし、Hugging Faceにログインします。

In [ ]:
# Google Colabでの要件のインストール
# !pip install transformers datasets trl huggingface_hub

# Hugging Faceへの認証
from huggingface_hub import login

login()

# 便利のため、Hugging Faceのトークンを.envファイルのHF_TOKENとして環境変数に設定できます

## 2. モデルとデータセットのロード

次に、事前学習済みモデルとデータセットをロードします。

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset

# モデルとトークナイザーをロード
model_name = "HuggingFaceTB/SmolLM2-135M"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# データセットをロード
dataset = load_dataset("Anthropic/hh-rlhf")

## 3. ORPOトレーナーの設定

ORPOトレーナーを設定し、トレーニングを開始します。

In [ ]:
from trl import ORPOConfig, ORPOTrainer

# ORPOの設定
orpo_config = ORPOConfig(
    model_name_or_path=model_name,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_steps=100,
    output_dir="./orpo_output"
)

# ORPOトレーナーを初期化
trainer = ORPOTrainer(
    model=model,
    args=orpo_config,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    tokenizer=tokenizer
)

# モデルをトレーニング
trainer.train()

## 4. 微調整されたモデルの保存

トレーニングが完了したら、微調整されたモデルを保存します。

In [ ]:
# 微調整されたモデルを保存
trainer.save_model("./orpo_finetuned_model")

## 5. 微調整されたモデルの評価

最後に、微調整されたモデルを評価します。

In [ ]:
# 評価の実行
results = trainer.evaluate()
print(results)